## Fine-tune a Bert model to identify specific topics

In [ ]:
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

import pandas as pd
LABEL_SCHEMA = {
    "impact_type": {
        "type": "single-label",
        "classes": [
            "death", "injury", "displacement", "missing", "disease",
            "economic_loss", "conflict", "weather_shock", "policy_change", "none"
        ]
    },
    "resource": {
        "type": "multi-label",
        "classes": [
            "food", "water", "cash_aid", "healthcare", "shelter",
            "livelihoods", "education", "infrastructure", "none"
        ]
    },
    "urgency": {
        "type": "single-label",
        "classes": [
            "low", "moderate", "high", "unclear"
        ]
    }
}

# Load the full labeled dataset
df = pd.read_csv("./for_bert/labeled_chunk_all.csv")

# Parse resource column from string to list
df["resource"] = df["resource"].apply(ast.literal_eval)

# Binarize the resource field
mlb = MultiLabelBinarizer()
resource_binary = mlb.fit_transform(df["resource"])
resource_classes = mlb.classes_

# Add one binary column per resource class
for i, label in enumerate(resource_classes):
    df[f"resource_{label}"] = resource_binary[:, i]

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
df["token_length"] = df["clean_text"].apply(lambda x: len(tokenizer.tokenize(str(x))))

# Combine fields for stratification
df["stratify_label"] = df["impact_type"] + "_" + df["urgency"]

# Split into train and validation sets
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["stratify_label"]
)

# Final feature columns
resource_cols = [f"resource_{label}" for label in resource_classes]
final_columns = ["clean_text", "impact_type", "urgency"] + resource_cols

train_df = train_df[final_columns].reset_index(drop=True)
val_df = val_df[final_columns].reset_index(drop=True)

# Token length stats
print("Token length stats (train):")
print(train_df["clean_text"].apply(lambda x: len(tokenizer.tokenize(str(x)))).describe())
print("\nTrain/Val shape:", train_df.shape, val_df.shape)

# -------------------------------
# Optimized chunking (no decoding)
# -------------------------------

def chunk_input_ids(text, tokenizer, max_length=512, stride=256):
    """
    Splits text into overlapping input_id chunks (no decoding).
    """
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return [tokens[i:i+max_length] for i in range(0, len(tokens), stride) if tokens[i:i+max_length]]

def expand_dataset_fast(df, text_col="clean_text", label_cols=None):
    """
    Applies chunking to every row and replicates labels per chunk.
    Stores input_ids (not raw text).
    """
    label_cols = label_cols or ["impact_type", "urgency"] + [c for c in df.columns if c.startswith("resource_")]
    records = []

    for row in tqdm(df.itertuples(), total=len(df)):
        chunks = chunk_input_ids(getattr(row, text_col), tokenizer)
        for chunk in chunks:
            record = { "input_ids": chunk }
            for col in label_cols:
                record[col] = getattr(row, col)
            records.append(record)

    return pd.DataFrame(records)

# Apply to both train and validation sets
expanded_train_df = expand_dataset_fast(train_df)
expanded_val_df = expand_dataset_fast(val_df)

# Final shapes
print("\nExpanded train shape:", expanded_train_df.shape)
print("Expanded val shape:", expanded_val_df.shape)

# Optional: save to CSV or pickle if needed

# Encode single-label classification targets
impact_enc = LabelEncoder()
urgency_enc = LabelEncoder()

expanded_train_df["impact_type_id"] = impact_enc.fit_transform(expanded_train_df["impact_type"])
expanded_val_df["impact_type_id"] = impact_enc.transform(expanded_val_df["impact_type"])

expanded_train_df["urgency_id"] = urgency_enc.fit_transform(expanded_train_df["urgency"])
expanded_val_df["urgency_id"] = urgency_enc.transform(expanded_val_df["urgency"])

expanded_train_df.to_pickle("./for_bert/expanded_train.pkl")
expanded_val_df.to_pickle("./for_bert/expanded_val.pkl")

In [ ]:
# ============================ 0. Imports & Device ============================
import os, torch, torch.nn as nn, pandas as pd, numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import Dataset as TorchDataset
from transformers import (
    BertModel, BertTokenizer, BertPreTrainedModel,
    DataCollatorWithPadding, TrainingArguments, Trainer
)
from transformers.modeling_outputs import SequenceClassifierOutput

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================ 1. Model =======================================
class MultiTaskBERT(BertPreTrainedModel):
    def __init__(self, config, num_impact, num_urgency, num_resource):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.1)
        self.impact_head   = nn.Linear(config.hidden_size, num_impact)
        self.urgency_head  = nn.Linear(config.hidden_size, num_urgency)
        self.resource_head = nn.Linear(config.hidden_size, num_resource)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        impact_type_id=None,
        urgency_id=None,
        **kwargs      # each resource_* label arrives here
    ):
        pooled = self.dropout(
            self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        )
        impact_logits   = self.impact_head(pooled)
        urgency_logits  = self.urgency_head(pooled)
        resource_logits = self.resource_head(pooled)

        loss = None
        if impact_type_id is not None and urgency_id is not None:
            # stack resource labels in consistent column order
            resource_labels = torch.stack(
                [kwargs[col] for col in sorted(k for k in kwargs if k.startswith("resource_"))],
                dim=1
            ).float()
            loss = (
                nn.CrossEntropyLoss()(impact_logits, impact_type_id) +
                nn.CrossEntropyLoss()(urgency_logits, urgency_id)   +
                0.5 * nn.BCEWithLogitsLoss()(resource_logits, resource_labels)
            )

        return SequenceClassifierOutput(
            loss   = loss,
            logits = (impact_logits, urgency_logits, resource_logits),
        )

# ============================ 2. Data Loading ================================
train_path = "/content/drive/MyDrive/phd/expanded_train.pkl"
val_path   = "/content/drive/MyDrive/phd/expanded_val.pkl"

df_train = pd.read_pickle(train_path).reset_index(drop=True)
df_val   = pd.read_pickle(val_path).reset_index(drop=True)

impact_enc  = LabelEncoder()
urgency_enc = LabelEncoder()
df_train["impact_type_id"] = impact_enc.fit_transform(df_train["impact_type"])
df_val["impact_type_id"]   = impact_enc.transform(df_val["impact_type"])
df_train["urgency_id"] = urgency_enc.fit_transform(df_train["urgency"])
df_val["urgency_id"]   = urgency_enc.transform(df_val["urgency"])

resource_cols = [c for c in df_train.columns if c.startswith("resource_")]

class CustomTorchDataset(TorchDataset):
    def __init__(self, df, resource_cols):
        self.input_ids = list(df["input_ids"])
        self.impact_type_id = list(df["impact_type_id"])
        self.urgency_id = list(df["urgency_id"])
        self.resource_matrix = df[resource_cols].values.astype("float32")

    def __getitem__(self, idx):
        example = {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "impact_type_id": torch.tensor(self.impact_type_id[idx]),
            "urgency_id": torch.tensor(self.urgency_id[idx]),
        }
        # add each resource_* label separately
        for i, col in enumerate(resource_cols):
            example[col] = torch.tensor(self.resource_matrix[idx, i])
        return example

    def __len__(self):
        return len(self.input_ids)

train_ds = CustomTorchDataset(df_train, resource_cols)
val_ds   = CustomTorchDataset(df_val,   resource_cols)

# ============================ 3. Tokenizer & Collator ========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
collator  = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8, return_tensors="pt")

# ============================ 4. Build / Compile Model =======================
model = MultiTaskBERT.from_pretrained(
    "bert-base-uncased",
    num_impact=len(impact_enc.classes_),
    num_urgency=len(urgency_enc.classes_),
    num_resource=len(resource_cols)
).to(device)
model = torch.compile(model)

# ============================ 5. Training Args ===============================
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/phd/bert_multitask_model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=5,
    logging_dir="/content/drive/MyDrive/phd/logs",
    report_to="none",
    remove_unused_columns=False,
    fp16=True,
    label_names=["impact_type_id", "urgency_id"] + resource_cols  # ← add this line

)

# ============================ 6. Metrics =====================================
LABEL_ORDER = ["impact_type_id", "urgency_id"] + resource_cols   # same order!

def compute_metrics(pred):
    # label_ids is a tuple in that order
    impact_labels, urgency_labels, *resource_cols_labels = pred.label_ids

    impact_logits, urgency_logits, resource_logits = pred.predictions

    impact_preds  = impact_logits.argmax(axis=1)
    urgency_preds = urgency_logits.argmax(axis=1)

    # stack resource labels / preds
    resource_labels = np.stack(resource_cols_labels, axis=1)
    resource_preds  = (torch.sigmoid(torch.tensor(resource_logits)) > 0.5).int().numpy()

    return dict(
        impact_acc       = accuracy_score(impact_labels, impact_preds),
        urgency_acc      = accuracy_score(urgency_labels, urgency_preds),
        resource_f1_micro= f1_score(resource_labels, resource_preds, average="micro"),
        resource_f1_macro= f1_score(resource_labels, resource_preds, average="macro"),
    )

# ============================ 7. Trainer =====================================
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics
)

# ---------- Resume if checkpoint exists ----------
ckpt_dir   = args.output_dir
ckpts = [os.path.join(ckpt_dir, d) for d in os.listdir(ckpt_dir) if d.startswith("checkpoint-")]
if ckpts:
    latest = max(ckpts, key=lambda p: int(p.split("-")[-1]))
    print("Resuming from:", latest)
    trainer.train(resume_from_checkpoint=latest)
else:
    trainer.train()

# ============================ 8. Save Final Model ============================
final_path = "/content/drive/MyDrive/phd/bert_multitask_model/final"
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)